In [1]:
import keras
import os
import numpy as np
import matplotlib.pyplot as plt
from keras import layers,models,optimizers
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator



In [ ]:
conv_base=VGG16(weights='imagenet',
                include_top=False,
                input_shape=(150,150,3))
conv_base.summary()

In [ ]:
base_dir='/kaggle/input'
train_dir=os.path.join(base_dir,'train')
validation_dir=os.path.join(base_dir,'validation')
test_dir=os.path.join(base_dir,'test1')

In [ ]:
datagen=ImageDataGenerator(rescale=1./255)
bath_size=20

In [ ]:
def extract_features(directory,sample_count):
    features=np.zeros(shape=(sample_count,4,4,512))
    labels=np.zeros(shape=(sample_count))
    generator=datagen.flow_from_directory(
                directory,
                target_size=(150,150),
                batch_size=bath_size,
                class_mode='binary'
    )
    i=0
    for input_batch,labels_batch in generator:
        features_batch=conv_base.predict(input_batch)
        features[i*bath_size:(i+1)*bath_size]=features_batch
        labels[i*bath_size:(i+1)*bath_size]=labels_batch
        i+=1
        if i*bath_size>=sample_count: break
    return features,labels

In [ ]:
train_features,train_labels=extract_features(train_dir,2000)
validation_features,validation_labels=extract_features(validation_dir,1000)
test_features,test_labels=extract_features(test_dir,1000)

In [ ]:
train_features=np.reshape(train_features,(2000,4*4*512))
validation_features=np.reshape(validation_features,(1000,4*4*512))
test_features=np.reshape(test_features,(1000,4*4*512))

In [ ]:
model=models.Sequential()
model.add(layers.Dense(256, activation='relu',input_dim=4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
                loss='binary_crossentropy',
                metrics=['acc'])

In [ ]:
history=model.fit(
    train_features,
    train_labels,
    epochs=30,
    batch_size=20,
    validation_data=(validation_features,validation_labels)
)
model.save('cat_and_dog_p2_1.h5')


In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

plt.plot(epochs,loss,'bo',label='Strata trenowania')
plt.plot(epochs,val_loss,'b',label='Strata walidacji')
plt.title('Strata trenowania i walidacji')
plt.legend()
plt.figure()

plt.plot(epochs,acc,'bo',label='Dokladnosc trenowania')
plt.plot(epochs,val_acc,'b',label='Dokladnosc walidacji')
plt.title('Dokladnosc trenowania i walidacji')

plt.show()



not on kaggle

In [ ]:
#nowy model 
model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest')

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

validation_generator=test_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),#'rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
history=model.fit(
    train_generator,
    # train_labels,
    steps_per_epoch=100,
    epochs=30,
    batch_size=20,
    validation_data=validation_generator,
    validation_steps=50,
    verbose=2
)
model.save('cat_and_dog_p2_2.h5')


In [ ]:
#zamrażanie 
conv_base.trainable=True
set_trainable=False
for layer in conv_base.layers:
    if layer.name=='block5_conv1':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),#'rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
history=model.fit(
    train_generator,
    # train_labels,
    steps_per_epoch=100,
    epochs=30,
    batch_size=20,
    validation_data=validation_generator,
    validation_steps=50,
    verbose=2
)
model.save('cat_and_dog_p2_3.h5')
